<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://raw.githubusercontent.com/dynovaio/open-source-office/refs/heads/master/assets/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://raw.githubusercontent.com/dynovaio/open-source-office/refs/heads/master/assets/dynova/dynova-logo-195x64.png" />
</div>

# Playground 1: NRQL Queries

## Imports

Python Imports

In [ ]:
import os

Third-party libraries

In [ ]:
import dotenv
import matplotlib.pyplot as plt
import pandas as pd
from newrelic_sb_sdk.client import NewRelicGqlClient
from newrelic_sb_sdk.graphql import nerdgraph
from newrelic_sb_sdk.graphql.objects import RootMutationType, RootQueryType
from newrelic_sb_sdk.graphql.scalars import Int
from newrelic_sb_sdk.utils.response import get_response_data, print_response
from sgqlc.operation import Operation
from sgqlc.types import Arg, Variable

## Client setup

To setup the client, first we need to open load the credentials from environment variables.

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

With environment varaibles loaded, we can proceed to instantiate the client

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key=NEW_RELIC_USER_KEY)

## Client testing

In order to use an test the client we need configure `query_type` and `mutation_type` for the `nerdgraph` schema

In [ ]:
nerdgraph.query_type = RootQueryType
nerdgraph.mutation_type = RootMutationType

For testing we will use a simple query in GraphQL to get the atttributes from our user

```graphql
query {
  actor {
    user {
      email
      id
      name
    }
  }
}
```

This query will be build from the `nerdgraph` schema.

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)

# Get the fields `id`, `email`, `name` from the `user` entity inside `actor`
operation.actor.user.__fields__("id", "email", "name")

This operation can be transformed into a GraphQL query

In [ ]:
query = operation.__to_graphql__()
print(query)

And this query is the one we send to be executed by our client.

In [ ]:
response = newrelic.execute(query)

The response data obtained is

In [ ]:
print_response(response)

We can also use the raw query directly written by hand. For this is recomendable to use the `build_query` method in order to get a clean query.

In [ ]:
query = newrelic.build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

And execute directly, as in the previous execution, with the same result

In [ ]:
response = newrelic.execute(query)
print_response(response)

## Playground area

### Doing NRQL with the API

#### Example 1

We can request New Relic data from the `Transaction` table calling the `nrql` action inside `actor`. For this we will use a generic NRQL query to use.

```nrql
FROM Transaction SELECT *
```

In [ ]:
template = """
    {
        actor {
            nrql(
                query: "SELECT * FROM Transaction",
                accounts: %(account_id)d,
            ) {
                nrql
                results
            }
        }
    }
"""

params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
results = get_response_data(
    response,
    key_path="nrql:results",
    action="actor",
)

In [ ]:
results_df = pd.DataFrame(
    data=results,
)

In [ ]:
results_df

#### Example 2

Now we goint to show a more elaborated example using query variables as usual in NRQL. For this example the NRQL query is

```nrql
FROM Transaction SELECT average(duration) FACET appName SINCE 1 hour ago TIMESERIES 1 minute
```

In [ ]:
template = """
    query ($accountId: Int!) {
        actor {
            nrql(
                query: "FROM Transaction SELECT average(duration) FACET appName SINCE 1 hour ago TIMESERIES 1 minute",
                accounts: $accountId,
            ) {
                nrql
                results
            }
        }
    }
"""

query = newrelic.build_query(template)
variables = {
    "accountId": int(NEW_RELIC_ACCOUNT_ID),
}

In [ ]:
response = newrelic.execute(query, variables=variables)

In [ ]:
results = get_response_data(
    response,
    key_path="nrql:results",
    action="actor",
)

In [ ]:
results_df = pd.DataFrame(
    data=results,
)

In [ ]:
results_df

In [ ]:
app_names = results_df["appName"].unique()

In [ ]:
app_names

In [ ]:
plt.clf()

fig = plt.figure(0, figsize=(10, 5))
axs = fig.add_subplot(1, 1, 1)

for app_name in app_names:
    data = results_df.query("appName == @app_name")
    data = data[["beginTimeSeconds", "average.duration"]]
    data["beginTimeSeconds"] = pd.to_datetime(
        data["beginTimeSeconds"],
        unit="s",
    )

    axs.plot(
        data["beginTimeSeconds"],
        data["average.duration"],
        label=app_name,
        marker=".",
    )

fig.legend(
    frameon=False,
    loc="lower center",
    ncols=3,
    bbox_to_anchor=(0.5, -0.1),
)

fig.tight_layout()

plt.show()

#### Example 3

In this example we will repeat the Example 3 using the `nerdgraph` schema as we did in the cluent tests.

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(
    nerdgraph.query_type,
    variables={
        "accountId": Arg(Int),  # this must match the my_query arg type!
    },
)

# Get the fields `id`, `email`, `name` from the `user` entity inside `actor`
operation.actor.nrql(
    query="FROM Transaction SELECT average(duration) FACET appName SINCE 1 hour ago TIMESERIES 1 minute",
    accounts=[
        Variable("accountId"),
    ],
).__fields__(
    "nrql",
    "results",
)

In [ ]:
query = operation.__to_graphql__()
variables = {
    "accountId": int(NEW_RELIC_ACCOUNT_ID),
}

In [ ]:
response = newrelic.execute(query, variables=variables)

In [ ]:
results = get_response_data(
    response,
    key_path="nrql:results",
    action="actor",
)

In [ ]:
results_df = pd.DataFrame(
    data=results,
)

In [ ]:
results_df

### It's your turn!

From now on you can use the client defined in the previous section to make your queries to the New Relic GraphQL API

In [ ]:
# Put your code here :D